<a href="https://colab.research.google.com/github/saranshikens/Epoch-Spring-Camp/blob/main/Automatic_Differentiaton.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**TASK-2 IMPLEMENTATION OF AUTOMATIC DIFFERENTIATION**  
By - Saransh

In [ ]:
import numpy as np

Definition of a Node Class, that will serve as nodes in the Computational Graph  


In [ ]:
class Node:
    def __init__(self, val, children=[], op=None, op_args=None):
        self.val = np.array(val)
        self.children = list(children)
        self.op = op
        self.op_args = op_args
        self.grad = 0

    def add(self, arg):
        if not isinstance(arg, Node): #Let's say our function is f(x,y,z) = x+2y+z. The constant 2 is not a node by default, so we convert it to one.
            arg = Node(arg)
        return Node(self.val+arg.val, children = [self, arg], op = "add")

    def sub(self, arg):
        if not isinstance(arg, Node):
            arg = Node(arg)
        return Node(self.val-arg.val, children = [self, arg], op = "sub")

    def mul(self, arg):
        if not isinstance(arg, Node):
            arg = Node(arg)
        return Node(self.val*arg.val, children = [self, arg], op = "mul")

    def div(self, arg):
        if not isinstance(arg, Node):
            arg = Node(arg)
        return Node(self.val/arg.val, children = [self, arg], op = "div")

    def pow(self, arg):
        if not isinstance(arg, Node):
            arg = Node(arg)
        return Node(self.val**arg.val, children = [self, arg], op = "pow")

    def backward(self, grad=1.0):
        self.grad += grad

        if self.op == "add":
           self.children[0].backward(grad)
           self.children[1].backward(grad)

        elif self.op == "sub":
             self.children[0].backward(grad)
             self.children[1].backward(-grad)

        elif self.op == "mul":
             self.children[0].backward(self.children[1].val*grad)
             self.children[1].backward(self.children[0].val*grad)

        elif self.op == "div":
             self.children[0].backward((1/self.children[1].val)*grad)
             self.children[1].backward(-(self.children[0].val/(self.children[1].val**2))*grad)

        elif self.op == "pow":
             self.children[0].backward(self.children[1].val*(self.children[0].val**(self.children[1].val-1))*grad)
             self.children[1].backward(self.children[0].val**self.children[1].val*np.log(self.children[0].val)*grad)


**EXPLANATION OF backward() METHOD**  
children[0] refers to 'x', whereas children[1] refers to 'y'.

1. op == "add"  
   Let $f(x,y) = 2x+3y$. Then,  
   $\frac{\partial z}{\partial x} = 2$, and $\frac{\partial z}{\partial y} = 3$.  
   As we can see, both children[0] and children[1] receive +grad, since both partial derivatives are positive.  
2. op == "sub"  
   Let $f(x,y) = 2x-3y$. Then,  
   $\frac{\partial z}{\partial x} = 2$, and $\frac{\partial z}{\partial y} = -3$.  
   Since $\frac{\partial z}{\partial x} > 0$ and $\frac{\partial z}{\partial x} < 0$, children[0] receives +grad, and children[1] receieves -grad.  
3. op == "mul"  
   Let $f(x,y) = z = xy$. Then,  
   $\frac{\partial z}{\partial x} = y$, and $\frac{\partial z}{\partial y} = x$.  
   As we can see, children[0] contributes children[1] to the grad, and vice-versa.  
4. op == "div"  
   Let $f(x,y) = z = \frac{x}{y}$. Then,  
   $\frac{\partial z}{\partial x} = \frac{1}{y}$, and $\frac{\partial z}{\partial y} = \frac{-x}{y^2}$.  
   So, children[0] contributes $\frac{1}{children[1]}$, and children[1] contributes $\frac{-children[0]}{children[1]^2}$ to the grad.  
5. op == "pow"  
   Let $f(x,y) = z = x^y$. Then,  
   $\frac{\partial z}{\partial x} = yx^{y-1}$, and $\frac{\partial z}{\partial y} = x^yln(x) = zln(x)$.  
   So, children[0] contributes $(children[1])(children[0]^{children[1]-1})$, and children[1] contributes $children[0]^{children[1]}ln(children[0])$ to the grad.  



**EXAMPLE INPUT FOR ARITHMETIC METHODS**

In [ ]:
x = Node(1)
y = Node(2)
z = Node(3)
b = Node.add(x, y)
a = Node.mul(b, z)
print(a.val)

9


**EXAMPLE INPUT FOR AUTOMATIC DIFFERENTIATION**

In [ ]:
A = Node(1)
B = Node(2)
temp = Node.add(A, A)
C = Node.add(Node.mul(A, 2), B)
D = Node.pow(C, 3)
D.backward()
print(f'Gradient of D wrt A: {A.grad}')
print(f'Gradient of D wrt B: {B.grad}')
print(f'Gradient of D wrt C: {C.grad}')

Gradient of D wrt A: 96.0
Gradient of D wrt B: 48.0
Gradient of D wrt C: 48.0
